In [1]:
import pandas as pd

In [81]:
data = pd.read_csv("twitter_training.csv", header=None, names=['id', 'topic', 'target', 'text'])
data = data[['id', 'topic', 'text', 'target']]
data = data[data['target'].isin(['Positive', 'Negative'])]
data['is_positive'] = (data['target'] == 'Positive').astype(int)
data.dropna(subset=['text', 'target'], inplace=True)

data.sample(5)

,id,topic,text,target,is_positive
237,2440,Borderlands,This is shitty. I get that profit-wise it had ...,Negative,0
43462,10260,PlayerUnknownsBattlegrounds(PUBG),I want to feel the the right way I felt today ...,Positive,1
3648,1831,CallOfDutyBlackopsColdWar,man i’m actually excited for Black Ops Cold Wa...,Positive,1
40311,1318,Battlefield,Dude lets hope next Battlefield brings back mo...,Positive,1
10374,12986,Xbox(Xseries),Holy shit it only took 2 console generations t...,Positive,1


### Режем датасет, чтобы уложиться в лимит апишки

In [89]:
data_cutted = data.sample(n=500,
                          random_state=0,
                          ignore_index=True)[['id', 'text', 'is_positive']]

In [90]:
data_cutted['is_positive'].value_counts()

is_positive
0    258
1    242
Name: count, dtype: int64

### Подключаем psycology AI апишку и проверяем, насколько хорошо она попадает

In [91]:
from first_api import call_first_api
import random

data_cutted_with_prediction = data_cutted.copy()
data_cutted_with_prediction['random_prediction'] = pd.Series()
data_cutted_with_prediction['first_api_prediction'] = pd.Series()

for idx in data_cutted_with_prediction.index:
    if idx%20 == 0:
        print(f"Start working with row {idx}")
    
    data_cutted_with_prediction.loc[idx, 'random_prediction'] = random.random()
    data_cutted_with_prediction.loc[idx, 'first_api_prediction'] = call_first_api(data_cutted_with_prediction.loc[idx, 'text'] )['pos']

data_cutted_with_prediction

Start working with row 0
Start working with row 20
Start working with row 40
Start working with row 60
Start working with row 80
Start working with row 100
Start working with row 120
Start working with row 140
Start working with row 160
Start working with row 180
Start working with row 200
Start working with row 220
Start working with row 240
Start working with row 260
Start working with row 280
Start working with row 300
Start working with row 320
Start working with row 340
Start working with row 360
Start working with row 380
Start working with row 400
Start working with row 420
Start working with row 440
Start working with row 460
Start working with row 480


,id,text,is_positive,random_prediction,first_api_prediction
0,785,Got Pred! Flew through plt 2 to dim 1 then we ...,0,0.738767,46.7
1,13177,Where is everyone sayin how big the xbox was a...,0,0.452282,65.4
2,9645,Then Amazing.,1,0.717913,97.8
3,3260,Left Instagram because no body shows me love. ...,0,0.423604,72.0
4,12758,"Seriously, is it hard to make a good remake? G...",0,0.003479,9.7
...,...,...,...,...,...
495,7875,@EAMaddenNFL just got me stuck on using a simp...,0,0.194692,48.8
496,9728,you lie the console is very sexy<unk>,1,0.926757,26.7
497,3878,God Fuck,0,0.957366,0.1
498,8052,"Xbox: ""No good exclusives, huh?""",0,0.125826,61.8


In [92]:
from sklearn.metrics import roc_auc_score
roc_auc_random = roc_auc_score(data_cutted_with_prediction['is_positive'],
                        data_cutted_with_prediction['random_prediction'])

roc_auc_first_api = roc_auc_score(data_cutted_with_prediction['is_positive'],
                        data_cutted_with_prediction['first_api_prediction'])

print(roc_auc_random, roc_auc_first_api)

0.47262797104234733 0.7803510795054135
